## Map generator
In dit bestand leg ik uit hoe ik maps genereer. Als eerste moet ik een algoritme kiezen om paden te genereren op een 2d map space, deze paden moeten een bepaalde dikte hebben.

- Bronnen:
    - https://theory.stanford.edu/~amitp/GameProgramming/AStarComparison.html
    - https://medium.com/@yvanscher/playing-with-perlin-noise-generating-realistic-archipelagos-b59f004d8401


Als eerste ben ik begonnen met een map genereren doormiddel van een noise functie, hiermee krijg ik een simpele map die wij als groepje kunnen gebruiken om alvast te proberen om de intergratie werkend te maken. De bedoeling is dat de map in een 2d array komt, deze array heeft verschillende nummers die geassiocieerd worden met verschillende tiles.

In [3]:
import noise
import numpy as np


In [ ]:
map_shape = (50, 100)

scale = 100
octaves = 6
persistence  = 0.5
lacunarity = 2.0

map = np.zeros(map_shape)
for i in range(map_shape[0]):
    for j in range(map_shape[1]):
        map[i][j] = noise.pnoise2(
            i / scale,
            j / scale,
            octaves=octaves,
            persistence=persistence,
            lacunarity=lacunarity,
            base=0
        )

TypeError: 'persistance' is an invalid keyword argument for noise2()